In [2]:
from google.colab import files
uploaded = files.upload()
import io
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['df_merged.csv']))

Saving df_merged.csv to df_merged (1).csv


In [3]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [10]:
data.head(3)

,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,單價元平方公尺,電梯,預售屋,建築完成距今,鄉鎮市區_中山區,...,移轉層次-數字,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_其他,建物型態_套房(1房1廳1衛),建物型態_店面(店鋪),建物型態_廠辦,建物型態_華廈(10層含以下有電梯),建物型態_辦公商業大樓,建物型態_透天厝
0,1,1,1,1,1,178092,0,0,26.0,0,...,5,1,0,0,0,0,0,0,0,0
1,5,4,3,1,1,139426,0,0,36.0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,1,1,1,0,106824,0,0,54.0,0,...,1,0,1,0,0,0,0,0,0,0


In [4]:
#區分特徵值與目標值
X = data.drop('單價元平方公尺', axis=1)
y = data['單價元平方公尺']
y /= 5000

In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
#這格要跑半小時
ratio = 100
ratiovalues = [i for i in range(10, ratio, 10)]
n_estimator = 250
n_estimatorvalues = [i for i in range(100, n_estimator, 50)]
depth = 10
depthvalues = [i for i in range(3, depth)]
rate = 2 
ratevalues = [0.1, 0.01, 0.001]
relative_best_train_score = 999999999999999999999
relative_best_test_score = 999999999999999999999
relative_best_ratio = 0
relative_best_estimators = 0
relative_best_depth = 0
relative_best_rate = 0
mse_trains = []
mse_tests = []

for k in ratiovalues:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=k/100, random_state=71)

    # 重置索引
    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)

    for l in n_estimatorvalues:
        for i in depthvalues:
            for j in ratevalues:
                reg = xgb.XGBRFRegressor(random_state=71, n_estimators=l, max_depth=i, learning_rate=j)
                reg.fit(X_train, y_train)
                y_pred_train = reg.predict(X_train)
                train_mse = mean_squared_error(y_train, y_pred_train)
                y_pred_test = reg.predict(X_test)
                test_mse = mean_squared_error(y_test, y_pred_test)
                mse_trains.append(train_mse)
                mse_tests.append(test_mse)

                #print("test_ratio:", k/100, "n_estimators:", l, "max_depth:", i, "learning_rate:", j, "Train MSE:", train_mse, "Test MSE:", test_mse)

                if train_mse < relative_best_train_score and test_mse < relative_best_test_score:
                    relative_best_train_score = train_mse
                    relative_best_test_score = test_mse
                    relative_best_ratio = k
                    relative_best_estimators = l
                    relative_best_depth = i
                    relative_best_rate = j

print("Best ratio of testing data:", relative_best_ratio)
print("Best no. of estimators:", relative_best_estimators)
print("Best depth:", relative_best_depth)
print("Best learning rate:", relative_best_rate)
print("Training MSE:", relative_best_train_score)
print("Testing MSE:", relative_best_test_score)

In [ ]:
plt.plot(ratiovalues, mse_trains, '-o', label='Train')
plt.plot(ratiovalues, mse_tests, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
plt.plot(n_estimatorvalues, mse_trains, '-o', label='Train')
plt.plot(n_estimatorvalues, mse_tests, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
plt.plot(depthvalues, mse_trains, '-o', label='Train')
plt.plot(depthvalues, mse_tests, '-o', label='Test')
plt.legend()
plt.show()

In [ ]:
plt.plot(ratiovalues, mse_trains, '-o', label='Train')
plt.plot(ratiovalues, mse_tests, '-o', label='Test')
plt.legend()
plt.show()

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=80/100, random_state=71)

# 重置索引
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

reg = xgb.XGBRFRegressor(random_state=71, n_estimators=100, max_depth=8, learning_rate=0.1)
reg.fit(X_train, y_train)
y_pred_train = reg.predict(X_train)
train_mse = mean_squared_error(y_train, y_pred_train)
y_pred_test = reg.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Training MSE:", train_mse)
print("Testing MSE:", test_mse)

Training MSE: 0.19552615731572318
Testing MSE: 0.2009336282728113


In [14]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.8, random_state=2023)
reg.fit(X_train2, y_train2)
perm = PermutationImportance(reg, random_state=2023).fit(X_test2, y_test2)
eli5.show_weights(perm, feature_names=X_train2.columns.tolist())

Weight,Feature
0.0033 ± 0.0002,建物型態_透天厝
0.0006 ± 0.0004,交易土地
0.0005 ± 0.0000,交易建物
0.0004 ± 0.0001,建物現況格局-房
0.0003 ± 0.0000,鄉鎮市區_文山區
0.0002 ± 0.0000,建物現況格局-衛
0.0002 ± 0.0000,鄉鎮市區_北投區
0.0001 ± 0.0000,鄉鎮市區_內湖區
0.0001 ± 0.0001,路段
0.0001 ± 0.0000,鄉鎮市區_士林區


In [16]:
importance_ranking = perm.feature_importances_

# Get the feature names
feature_names = X_train2.columns.tolist()

# Sort the feature names based on the importance ranking
importance_ranking_sorted = [feature for _, feature in sorted(zip(importance_ranking, feature_names), reverse=True)]

print("Feature Importance Ranking:", importance_ranking_sorted)

Feature Importance Ranking: ['建物型態_透天厝', '交易土地', '交易建物', '建物現況格局-房', '鄉鎮市區_文山區', '建物現況格局-衛', '鄉鎮市區_北投區', '鄉鎮市區_內湖區', '路段', '鄉鎮市區_士林區', '建物型態_其他', '交易距今', '建物現況格局-廳', '建物型態_住宅大樓(11層含以上有電梯)', '鄉鎮市區_萬華區', '電梯', '鄉鎮市區_南港區', '建物型態_辦公商業大樓', '建物型態_廠辦', '建物型態_套房(1房1廳1衛)', '建物型態_店面(店鋪)', '鄉鎮市區_大同區', '建物型態_華廈(10層含以下有電梯)', '建物現況格局-隔間', '有無管理組織', '鄉鎮市區_中山區', '鄉鎮市區_大安區', '預售屋', '建物型態_公寓(5樓含以下無電梯)', '交易車位', '鄉鎮市區_中正區', '鄉鎮市區_松山區', '鄉鎮市區_信義區', '移轉層次-數字', '建築完成距今']


In [19]:
# 設定要測試的最大特徵數量
max_num_features = len(importance_ranking_sorted)

# 初始化最佳模型相關變數
best_mse = float('inf')
best_num_features = 0
best_model = None

# 迴圈測試每個特徵數量
for num_features in range(1, max_num_features + 1):
    # 從 importance_ranking 中取出前 num_features 個特徵名稱
    selected_features = importance_ranking_sorted[:num_features]

    # 根據選定的特徵名稱從原始資料集 X 中取出對應的特徵
    selected_X = X[selected_features]

    # 分割訓練集和測試集
    X_train3, X_test3, y_train3, y_test3 = train_test_split(selected_X, y, test_size=0.8, random_state=71)

    # 建立並訓練 XGBoost 模型
    reg = xgb.XGBRFRegressor(random_state=71, n_estimators=100, max_depth=8, learning_rate=0.1)
    reg.fit(X_train3, y_train3)

    # 進行預測
    y_pred = reg.predict(X_test3)

    # 計算 MSE
    mse = mean_squared_error(y_test3, y_pred)

    # 判斷是否為最佳模型
    if mse < best_mse:
        best_mse = mse
        best_num_features = num_features
        best_model = reg

    print("特徵數量:", num_features, "MSE:", mse)

print("最佳特徵數量:", best_num_features)
print("最佳模型的 MSE:", best_mse)
print("最佳模型:", best_model)


特徵數量: 1 MSE: 0.2009410565334293
特徵數量: 2 MSE: 0.20094018180860726
特徵數量: 3 MSE: 0.20094049540406736
特徵數量: 4 MSE: 0.20094006730736969
特徵數量: 5 MSE: 0.2009402013228984
特徵數量: 6 MSE: 0.20094037831820924
特徵數量: 7 MSE: 0.20093969159465194
特徵數量: 8 MSE: 0.20094029993202728
特徵數量: 9 MSE: 0.20093981057922847
特徵數量: 10 MSE: 0.20093914657585782
特徵數量: 11 MSE: 0.20093878942268434
特徵數量: 12 MSE: 0.20093905456324826
特徵數量: 13 MSE: 0.200939076061107
特徵數量: 14 MSE: 0.20093929933663443
特徵數量: 15 MSE: 0.20093965678754963
特徵數量: 16 MSE: 0.20093845249168163
特徵數量: 17 MSE: 0.20093844804341982
特徵數量: 18 MSE: 0.2009385303927029
特徵數量: 19 MSE: 0.2009385095839208
特徵數量: 20 MSE: 0.2009386069013343
特徵數量: 21 MSE: 0.20093850267564473
特徵數量: 22 MSE: 0.20093873457676467
特徵數量: 23 MSE: 0.20093890214544688
特徵數量: 24 MSE: 0.20093909778779226
特徵數量: 25 MSE: 0.20093925887460043
特徵數量: 26 MSE: 0.20093870450988793
特徵數量: 27 MSE: 0.20093459759323823
特徵數量: 28 MSE: 0.20093457802341486
特徵數量: 29 MSE: 0.2009351069446155
特徵數量: 30 MSE: 0.200935821925714